In [46]:
import sqlite3
import pandas as pd

database = 'database.sqlite'
conn = sqlite3.connect(database)

cursor = conn.cursor()

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables


,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


1)Rescatando el ID de cada jugador chileno

In [47]:
#Tabla Player
Player = pd.read_sql_query(
'''select
*
from Player''', conn)

Jugadores_chilenos=['Alexis Sanchez','Gary Medel',
'Mauricio Isla',
'Arturo Vidal',
'Claudio Bravo',
'Charles Aranguiz',
'Gonzalo Jara',
'Jean Beausejour',
'Marcelo Diaz',
'Eduardo Vargas']

Jugadores_chilenos=pd.DataFrame(Jugadores_chilenos)
Jugadores_chilenos.columns = ['player_name']

df= pd.merge(Jugadores_chilenos, Player, on='player_name')
df=df.ix[:,'player_name':'player_api_id']
df.head(10)

#Jugadores_id=df['player_api_id']
#Jugadores_id



/Users/soledadrodriguez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,player_name,id,player_api_id
0,Alexis Sanchez,501,50047
1,Gary Medel,3691,49970
2,Mauricio Isla,7222,109298
3,Arturo Vidal,1013,49939
4,Claudio Bravo,1859,37421
5,Charles Aranguiz,1658,50638
6,Gonzalo Jara,3937,49885
7,Jean Beausejour,4759,38254
8,Marcelo Diaz,6637,49847
9,Eduardo Vargas,2891,50661


In [49]:
#Tabla Match
Match = pd.read_sql_query(
'''select
*
from Match''', conn)

columnas_match=Match.columns.tolist()

    

2)Explorando las columnas de la tabla Match

In [9]:
columnas_match

['id',
 'country_id',
 'league_id',
 'season',
 'stage',
 'date',
 'match_api_id',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player

3) Rescatando por posición la participación de los jugadores (home o away). Esto se hizo agrupando las columnas de las posiciones por ID jugador y contando la cantidad de apariciones de cada jugador en esa podición. Luego se cruzó esta información con la tabla que contiene los ID de los jugadores chilenos. Por último se sumaron horizontalmente las columnas de participación para obtener el total de partidos. 

In [51]:
dam=df
i=1
while i <= 11:
    
    hp=Match.groupby(['home_player_{}'.format(str(i))]).size().reset_index(name='counts_hp{}'.format(str(i)))
    hp.rename(columns={'home_player_{}'.format(str(i)): 'player_api_id'}, inplace=True)
    
    ap=Match.groupby(['away_player_{}'.format(str(i))]).size().reset_index(name='counts_ap{}'.format(str(i)))
    ap.rename(columns={'away_player_{}'.format(str(i)): 'player_api_id'}, inplace=True)
    
    df_hp=pd.merge(dam,hp,how='left',on='player_api_id')
    df_count=pd.merge(df_hp,ap,how='left',on='player_api_id')
    dam=df_count
    i+=1
    

dam=dam.drop(['id'], axis=1)
suma=dam.drop(['player_name','player_api_id'], axis=1)
suma['Total'] = suma.sum(axis=1)
partidos_por_jugador=suma['Total']
partidos_por_jugador= pd.concat([df, partidos_por_jugador], axis=1)
partidos_por_jugador.head(10)


,player_name,id,player_api_id,Total
0,Alexis Sanchez,501,50047,197.0
1,Gary Medel,3691,49970,172.0
2,Mauricio Isla,7222,109298,174.0
3,Arturo Vidal,1013,49939,220.0
4,Claudio Bravo,1859,37421,212.0
5,Charles Aranguiz,1658,50638,5.0
6,Gonzalo Jara,3937,49885,39.0
7,Jean Beausejour,4759,38254,57.0
8,Marcelo Diaz,6637,49847,60.0
9,Eduardo Vargas,2891,50661,45.0


4) Obtener mediante SQL, esto se hubiese logrado con FULL JOIN pero no es posible en sqlite

In [67]:
#####SQL

hp1 = pd.read_sql_query('''SELECT * FROM (SELECT home_player_1, COUNT(Id) FROM Match GROUP BY home_player_1) a JOIN (SELECT home_player_2, COUNT(Id) as hp2_count  FROM Match GROUP BY home_player_1) b WHERE a.home_player_1 = b.home_player_2 ;''', conn)
hp1


#hp1 = pd.read_sql_query('''SELECT *
#FROM (SELECT COUNT(Id) as hp1_count, home_player_1 FROM Match GROUP BY home_player_1) a
#LEFT JOIN (SELECT COUNT(Id) as hp2_count, home_player_2 FROM Match GROUP BY home_player_1) b WHERE a.home_player_1 = b.home_player_2
#UNION 
#SELECT *
#FROM (SELECT COUNT(Id) as hp2_count, home_player_2 FROM Match GROUP BY home_player_1) b
#LEFT JOIN (SELECT COUNT(Id) as hp1_count, home_player_1 FROM Match GROUP BY home_player_1) a WHERE a.home_player_1 = b.home_player_2
# ;''', conn)
#hp1



,home_player_1,COUNT(Id),home_player_2,hp2_count
0,26150,16,26150,1
1,37971,31,37971,3
2,40709,87,40709,1
3,41097,145,41097,1
4,43006,3,43006,1
5,107806,43,107806,1


5) Rescatando la cantidad de goles por jugador según su ID jugador 

In [52]:
goles=[]

goal_alexis=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>50047</%'; ''', conn)
goles.append(goal_alexis)
goal_gary=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>49970</%'; ''', conn)
goles.append(goal_gary)
goal_isla=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>109298</%'; ''', conn)
goles.append(goal_isla)
goal_vidal=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>49939</%'; ''', conn)
goles.append(goal_vidal)
goal_bravo=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>37421</%'; ''', conn)
goles.append(goal_bravo)
goal_charles=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>50638</%'; ''', conn)
goles.append(goal_charles)
goal_jara=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>49885</%'; ''', conn)
goles.append(goal_jara)
goal_beausejeur=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>38254</%'; ''', conn)
goles.append(goal_beausejeur)
goal_diaz=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>49847</%'; ''', conn)
goles.append(goal_diaz)
goal_varas=pd.read_sql_query(
'''select
count(goal)
from Match where Match.goal!= 'none' and Match.goal LIKE '%>50661</%'; ''', conn)
goles.append(goal_varas)



5)Consolidando la cantidad de goles por jugador en un mismo dataframe

In [61]:
#TABLA GOLES
columna_goles=pd.concat(goles,axis=0)
columna_goles=columna_goles.reset_index()

6) Tabla de consolidado final con nombre del jugador, ID del jugador, Cantidad de partidos y cantidad de goles

In [66]:
#TABLA FINAL

tabla_final=pd.concat([partidos_por_jugador,columna_goles],axis=1)
tabla_final=tabla_final.drop(['id','index'],axis=1)
tabla_final.rename({'player_name':'Nombre jugador','player_api_id':'ID jugador','Total': 'Cantidad de partidos', 'count(goal)': 'Cantidad de goles'}, axis=1) 

,Nombre jugador,ID jugador,Cantidad de partidos,Cantidad de goles
0,Alexis Sanchez,50047,197.0,85
1,Gary Medel,49970,172.0,12
2,Mauricio Isla,109298,174.0,20
3,Arturo Vidal,49939,220.0,63
4,Claudio Bravo,37421,212.0,0
5,Charles Aranguiz,50638,5.0,3
6,Gonzalo Jara,49885,39.0,3
7,Jean Beausejour,38254,57.0,16
8,Marcelo Diaz,49847,60.0,1
9,Eduardo Vargas,50661,45.0,12
